In [1800]:
# import required libraries

import numpy as np
import pandas as pd
import copy

In [1801]:
# Reading data from loan approval csv into python lists

data=pd.read_csv("loan_approval_dataset.csv")

# check the columns in csv
data.columns = data.columns.str.strip()
# print(data.columns)

X_original = data[[
    'no_of_dependents',
    'education',
    'self_employed',
    'income_annum',
    'loan_amount',
    'loan_term',
    'cibil_score',
    'residential_assets_value',
    'commercial_assets_value',
    'luxury_assets_value',
    'bank_asset_value',
]].values.tolist()

X = X_original

# code below normalises data
X = []
for row in X_original:
    row[1] = 10 if row[1].strip().lower() == 'graduate' else 0
    row[2] = 10 if row[2].strip().lower() == 'yes' else 0
    row_new = []
    for i in range(0, len(row)):
        if (i < 1 or i > 2):
            row_new.append(row[i] / X_original[0][i] * 10)
        else:
            row_new.append(row[i])
    X.append(row_new)

# reading loan approval status into Y as 1 or 0
Y = data["loan_status"].values.tolist()
Y = list(map(lambda st: 1 if st.strip() == 'Approved' else 0, Y))

# make X, Y numpy arrays for vectorisation
X = np.array(X)
X = X.reshape(11, X.shape[0])

Y = np.array(Y)
Y = Y.reshape(1, Y.shape[0])

In [1802]:
# defining sigmoid function

def sigmoid(Z):

    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

In [1803]:
# sigmoid backward function

def sigmoid_backward(dA, cache):

    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ

In [1804]:
# defining relu function

def relu(Z):
    A = np.maximum(0, Z)

    assert(A.shape == Z.shape)
    cache = Z
    return A, cache

In [1805]:
# sigmoid backward function

def relu_backward(dA, cache):

    Z = cache
    dZ = np.array(dA, copy=True)

    for i in range(len(dZ)):
        for j in range(len(dZ[i])):
            dZ[i][j]  = max(Z[i][j], 0)
    
    return dZ

In [1806]:
# initialising the parameters

def initialise_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)

    for i in range(1, L):
        parameters['W' + str(i)] = np.random.rand(layer_dims[i], layer_dims[i - 1]) * .001
        parameters['b' + str(i)] = np.zeros((layer_dims[i], 1))

    return parameters

In [1807]:
# linear forward function

def linear_forward(A, W, b):

    Z = np.dot(W, A) + b
    cache = (A, W, b)

    return Z, cache 

In [1808]:
# linear activation forward

def linear_activation_forward(A_prev, W, b, activation):
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)

    return A, cache

In [1809]:
# complete forward pass

def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2

    for i in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(i)], parameters['b' + str(i)], activation='relu' )
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation='sigmoid' )
    caches.append(cache)

    return AL, caches

In [1810]:
# cost function

def compute_cost(AL, Y):
    m = Y.shape[1]

    cost = (-1/m) * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
    
    cost = np.squeeze(cost)
    return cost

In [1811]:
# linear backward

def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)

    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

In [1812]:
# linear activation backward

def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    if activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [1813]:
# complete forward pass

def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L - 1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation = 'sigmoid')
    grads['dA' + str(L - 1)] = dA_prev_temp
    grads['dW' + str(L)] = dW_temp
    grads['db' + str(L)] = db_temp

    for i in reversed(range(L - 1)):
        current_cache = caches[i]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads['dA' + str(i + 1)], current_cache, activation = 'relu')
        grads['dA' + str(i)] = dA_prev_temp
        grads['dW' + str(i + 1)] = dW_temp
        grads['db' + str(i + 1)] = db_temp
    
    return grads

In [1814]:
# update parameters

def update_parameters(params, grads, size, learning_rate):
    parameters = copy.deepcopy(params)
    L = size

    for i in range(L):
        parameters['W' + str(i + 1)] = parameters['W' + str(i + 1)] - learning_rate * grads['dW' + str(i + 1)]
        parameters['b' + str(i + 1)] = parameters['b' + str(i + 1)] - learning_rate * grads['db' + str(i + 1)]
    
    return parameters

In [1815]:
# deep neural network model

def L_layer_model(X, Y, layer_dims):

    learning_rate = 0.0075
    num_iterations = 300

    np.random.seed(1)
    costs = []

    parameters = initialise_parameters_deep(layer_dims)

    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)

        cost = compute_cost(AL, Y)
        costs.append(cost)

        grads = L_model_backward(AL, Y, caches)

        parameters = update_parameters(parameters, grads, len(layer_dims) - 1, learning_rate)

        print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))

    return parameters, costs

In [1816]:
layer_dims = [11,4,3,2,1]
parameters, costs = L_layer_model(X, Y , layer_dims)

Cost after iteration 0: 0.6931471805441671
Cost after iteration 1: 0.6930353629417604
Cost after iteration 2: 0.6929239642464395
Cost after iteration 3: 0.6928129829043067
Cost after iteration 4: 0.6927024173513647
Cost after iteration 5: 0.6925922660293087
Cost after iteration 6: 0.6924825273855058
Cost after iteration 7: 0.6923731998729772
Cost after iteration 8: 0.6922642819503804
Cost after iteration 9: 0.69215577208199
Cost after iteration 10: 0.69204766873768
Cost after iteration 11: 0.6919399703929051
Cost after iteration 12: 0.6918326755286828
Cost after iteration 13: 0.6917257826315755
Cost after iteration 14: 0.6916192901936714
Cost after iteration 15: 0.6915131967125673
Cost after iteration 16: 0.6914075006913496
Cost after iteration 17: 0.6913022006385774
Cost after iteration 18: 0.6911972950682639
Cost after iteration 19: 0.6910927824998582
Cost after iteration 20: 0.6909886614582278
Cost after iteration 21: 0.6908849304736403
Cost after iteration 22: 0.6907815880817458
Co

In [1817]:
def predict(X_new, y_new):

    m = X.shape[1]
    n = len(parameters) // 2
    p = np.zeros((1, m))

    probas, caches = L_model_forward(X_new, parameters)

    for i in range(0, probas.shape[1]):
        if probas[0, i] > 0.5:
            p[0, i] = 1
        else:
            p[0, i] = 0

    print("Accuracy = " + str(np.sum(p == y_new)/m))

In [1818]:
x_new = np.array([0.0, 0, 10, 4.270833333333333, 4.080267558528428, 6.666666666666666, 5.359897172236504, 11.25, 1.25, 3.8766519823788546, 4.125])

predict(x_new, 0)

Accuracy = 0.9990630124150855
